# Data Wrangling & Analyzing Twitter Data

## *WeRateDogs project*

### Saeed Falana
### Ramallah, Plaestine


## Introduction

Through this project will gather data from a variety of sources and in a variety of formats, assess its quality and tidiness, then clean it.

The dataset that we will be wrangling (and analyzing and visualizing) is the tweet archive of Twitter user [@dog_rates](https://twitter.com/dog_rates), also known as [WeRateDogs](https://en.wikipedia.org/wiki/WeRateDogs).

## Gather

Gathering Data for this Project composed from three pieces of data as described below:

- The WeRateDogs Twitter archive. We will download this file manually by clicking the following link:  [twitter_archive_enhanced.csv](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv)
- The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (image_predictions.tsv) hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv
- Each tweet's retweet count and favorite (i.e. "like") count at minimum, and any additional data we will find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, we will query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then we will read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.

** Let's Start **

In [1]:
# Import the libraries that we will need in this project
import pandas as pd
import datetime as dt
import numpy as np
import requests
import tweepy
import json
import re
import time
from nltk import pos_tag

In [4]:
# Read the twitter-archive-enhanced.csv file and store it as dataframe in archive
archive = pd.read_csv('twitter-archive-enhanced.csv', encoding = 'utf-8')
# Quick check to the file content and structure
archive.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


In [5]:
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

** The file twitter-archive-enhanced.csv successfully stored in archive data frame, it has 17 columns and 2356 entries **

In [6]:
# Using Requests library to download a file then store it in a tsv file
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open(url.split('/')[-1], mode = 'wb') as outfile:
    outfile.write(response.content)

# Read the downloaded file into a dataframe 'images'
images = pd.read_csv('image-predictions.tsv', sep = '\t', encoding = 'utf-8')
# Quick check to the file content and structure
images

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [7]:
images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


** The file image-predictions.tsv successfully downloaded and stored in images data frame, it has 12 columns and 2075 entries **

In [10]:
#Importing libraries
import tweepy
from tweepy import OAuthHandler
import json
import csv
import sys
import os
import time
# authentication pieces
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth_handler=auth, 
                 wait_on_rate_limit=True, 
                 wait_on_rate_limit_notify=True)

In [11]:
tweet_ids = list(archive.tweet_id)

tweet_data = {}
for tweet in tweet_ids:
    try:
        tweet_status = api.get_status(tweet,
                                      wait_on_rate_limit=True, 
                                      wait_on_rate_limit_notify=True)
        tweet_data[str(tweet)] = tweet_status._json
    except: 
        print("Error for: " + str(tweet))

Error for: 892420643555336193
Error for: 892177421306343426
Error for: 891815181378084864
Error for: 891689557279858688
Error for: 891327558926688256
Error for: 891087950875897856
Error for: 890971913173991426
Error for: 890729181411237888
Error for: 890609185150312448
Error for: 890240255349198849
Error for: 890006608113172480
Error for: 889880896479866881
Error for: 889665388333682689
Error for: 889638837579907072
Error for: 889531135344209921
Error for: 889278841981685760
Error for: 888917238123831296
Error for: 888804989199671297
Error for: 888554962724278272
Error for: 888202515573088257
Error for: 888078434458587136
Error for: 887705289381826560
Error for: 887517139158093824
Error for: 887473957103951883
Error for: 887343217045368832
Error for: 887101392804085760
Error for: 886983233522544640
Error for: 886736880519319552
Error for: 886680336477933568
Error for: 886366144734445568
Error for: 886267009285017600
Error for: 886258384151887873
Error for: 886054160059072513
Error for:

Error for: 840698636975636481
Error for: 840696689258311684
Error for: 840632337062862849
Error for: 840370681858686976
Error for: 840268004936019968
Error for: 839990271299457024
Error for: 839549326359670784
Error for: 839290600511926273
Error for: 839239871831150596
Error for: 838952994649550848
Error for: 838921590096166913
Error for: 838916489579200512
Error for: 838831947270979586
Error for: 838561493054533637
Error for: 838476387338051585
Error for: 838201503651401729
Error for: 838150277551247360
Error for: 838085839343206401
Error for: 838083903487373313
Error for: 837820167694528512
Error for: 837482249356513284
Error for: 837471256429613056
Error for: 837366284874571778
Error for: 837110210464448512
Error for: 837012587749474308
Error for: 836989968035819520
Error for: 836753516572119041
Error for: 836677758902222849
Error for: 836648853927522308
Error for: 836397794269200385
Error for: 836380477523124226
Error for: 836260088725786625
Error for: 836001077879255040
Error for:

Error for: 805487436403003392
Error for: 805207613751304193
Error for: 804738756058218496
Error for: 804475857670639616
Error for: 804413760345620481
Error for: 804026241225523202
Error for: 803773340896923648
Error for: 803692223237865472
Error for: 803638050916102144
Error for: 803380650405482500
Error for: 803321560782307329
Error for: 803276597545603072
Error for: 802952499103731712
Error for: 802624713319034886
Error for: 802600418706604034
Error for: 802572683846291456
Error for: 802323869084381190
Error for: 802265048156610565
Error for: 802247111496568832
Error for: 802239329049477120
Error for: 802185808107208704
Error for: 801958328846974976
Error for: 801854953262350336
Error for: 801538201127157760
Error for: 801285448605831168
Error for: 801167903437357056
Error for: 801127390143516673
Error for: 801115127852503040
Error for: 800859414831898624
Error for: 800855607700029440
Error for: 800751577355128832
Error for: 800513324630806528
Error for: 800459316964663297
Error for:

Error for: 770093767776997377
Error for: 770069151037685760
Error for: 769940425801170949
Error for: 769695466921623552
Error for: 769335591808995329
Error for: 769212283578875904
Error for: 768970937022709760
Error for: 768909767477751808
Error for: 768855141948723200
Error for: 768609597686943744
Error for: 768596291618299904
Error for: 768554158521745409
Error for: 768473857036525572
Error for: 768193404517830656
Error for: 767884188863397888
Error for: 767754930266464257
Error for: 767500508068192258
Error for: 767191397493538821
Error for: 767122157629476866
Error for: 766864461642756096
Error for: 766793450729734144
Error for: 766714921925144576
Error for: 766693177336135680
Error for: 766423258543644672
Error for: 766313316352462849
Error for: 766078092750233600
Error for: 766069199026450432
Error for: 766008592277377025
Error for: 765719909049503744
Error for: 765669560888528897
Error for: 765395769549590528
Error for: 765371061932261376
Error for: 765222098633691136
Error for:

Error for: 736365877722001409
Error for: 736225175608430592
Error for: 736010884653420544
Error for: 735991953473572864
Error for: 735648611367784448
Error for: 735635087207878657
Error for: 735274964362878976
Error for: 735256018284875776
Error for: 735137028879360001
Error for: 734912297295085568
Error for: 734787690684657664
Error for: 734776360183431168
Error for: 734559631394082816
Error for: 733828123016450049
Error for: 733822306246479872
Error for: 733482008106668032
Error for: 733460102733135873
Error for: 733109485275860992
Error for: 732732193018155009
Error for: 732726085725589504
Error for: 732585889486888962
Error for: 732375214819057664
Error for: 732005617171337216
Error for: 731285275100512256
Error for: 731156023742988288
Error for: 730924654643314689
Error for: 730573383004487680
Error for: 730427201120833536
Error for: 730211855403241472
Error for: 730196704625098752
Error for: 729854734790754305
Error for: 729838605770891264
Error for: 729823566028484608
Error for:

Error for: 702321140488925184
Error for: 702276748847800320
Error for: 702217446468493312
Error for: 701981390485725185
Error for: 701952816642965504
Error for: 701889187134500865
Error for: 701805642395348998
Error for: 701601587219795968
Error for: 701570477911896070
Error for: 701545186879471618
Error for: 701214700881756160
Error for: 700890391244103680
Error for: 700864154249383937
Error for: 700847567345688576
Error for: 700796979434098688
Error for: 700747788515020802
Error for: 700518061187723268
Error for: 700505138482569216
Error for: 700462010979500032
Error for: 700167517596164096
Error for: 700151421916807169
Error for: 700143752053182464
Error for: 700062718104104960
Error for: 700029284593901568
Error for: 700002074055016451
Error for: 699801817392291840
Error for: 699788877217865730
Error for: 699779630832685056
Error for: 699775878809702401
Error for: 699691744225525762
Error for: 699446877801091073
Error for: 699434518667751424
Error for: 699423671849451520
Error for:

Error for: 683852578183077888
Error for: 683849932751646720
Error for: 683834909291606017
Error for: 683828599284170753
Error for: 683773439333797890
Error for: 683742671509258241
Error for: 683515932363329536
Error for: 683498322573824003
Error for: 683481228088049664
Error for: 683462770029932544
Error for: 683449695444799489
Error for: 683391852557561860
Error for: 683357973142474752
Error for: 683142553609318400
Error for: 683111407806746624
Error for: 683098815881154561
Error for: 683078886620553216
Error for: 683030066213818368
Error for: 682962037429899265
Error for: 682808988178739200
Error for: 682788441537560576
Error for: 682750546109968385
Error for: 682697186228989953
Error for: 682662431982772225
Error for: 682638830361513985
Error for: 682429480204398592
Error for: 682406705142087680
Error for: 682393905736888321
Error for: 682389078323662849
Error for: 682303737705140231
Error for: 682259524040966145
Error for: 682242692827447297
Error for: 682088079302213632
Error for:

Error for: 674271431610523648
Error for: 674269164442398721
Error for: 674265582246694913
Error for: 674262580978937856
Error for: 674255168825880576
Error for: 674082852460433408
Error for: 674075285688614912
Error for: 674063288070742018
Error for: 674053186244734976
Error for: 674051556661161984
Error for: 674045139690631169
Error for: 674042553264685056
Error for: 674038233588723717
Error for: 674036086168010753
Error for: 674024893172875264
Error for: 674019345211760640
Error for: 674014384960745472
Error for: 674008982932058114
Error for: 673956914389192708
Error for: 673919437611909120
Error for: 673906403526995968
Error for: 673887867907739649
Error for: 673716320723169284
Error for: 673715861853720576
Error for: 673711475735838725
Error for: 673709992831262724
Error for: 673708611235921920
Error for: 673707060090052608
Error for: 673705679337693185
Error for: 673700254269775872
Error for: 673697980713705472
Error for: 673689733134946305
Error for: 673688752737402881
Error for:

Error for: 668932921458302977
Error for: 668902994700836864
Error for: 668892474547511297
Error for: 668872652652679168
Error for: 668852170888998912
Error for: 668826086256599040
Error for: 668815180734689280
Error for: 668779399630725120
Error for: 668655139528511488
Error for: 668645506898350081
Error for: 668643542311546881
Error for: 668641109086707712
Error for: 668636665813057536
Error for: 668633411083464705
Error for: 668631377374486528
Error for: 668627278264475648
Error for: 668625577880875008
Error for: 668623201287675904
Error for: 668620235289837568
Error for: 668614819948453888
Error for: 668587383441514497
Error for: 668567822092664832
Error for: 668544745690562560
Error for: 668542336805281792
Error for: 668537837512433665
Error for: 668528771708952576
Error for: 668507509523615744
Error for: 668496999348633600
Error for: 668484198282485761
Error for: 668480044826800133
Error for: 668466899341221888
Error for: 668297328638447616
Error for: 668291999406125056
Error for:

In [12]:
# Create DataFrames from list of dictionaries
json_tweets = pd.DataFrame(tweet_data, columns = ['tweet_id', 'favorites', 'retweets',
# Save the dataFrame in file
json_tweets.to_csv('tweet_json.txt', encoding = 'utf-8', index=False)

In [14]:
# Read the saved tweet_json.txt file into a dataframe
json_tweets = pd.read_csv('tweet_json.txt', encoding = 'utf-8')
json_tweets

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time
0,892420643555336193,39361,8793,4484668,124451,2017-08-01 16:23:56
1,892177421306343426,33693,6447,4484669,124451,2017-08-01 00:17:27
2,891815181378084864,25383,4272,4484669,124451,2017-07-31 00:18:03
3,891689557279858688,42731,8880,4484669,124451,2017-07-30 15:58:51
4,891327558926688256,40897,9661,4484669,124451,2017-07-29 16:00:24
5,891087950875897856,20496,3217,4484669,124451,2017-07-29 00:08:17
6,890971913173991426,12026,2130,4484669,124451,2017-07-28 16:27:12
7,890729181411237888,66500,19466,4484669,124451,2017-07-28 00:22:40
8,890609185150312448,28129,4370,4484669,124451,2017-07-27 16:25:51
9,890240255349198849,32365,7632,4484669,124451,2017-07-26 15:59:51


In [15]:
json_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 6 columns):
tweet_id           2356 non-null int64
favorites          2356 non-null int64
retweets           2356 non-null int64
user_followers     2356 non-null int64
user_favourites    2356 non-null int64
date_time          2356 non-null object
dtypes: int64(5), object(1)
memory usage: 110.5+ KB


** The file tweet_json.txt successfully saved in our working directory contains the result of the API Querying then stored in json_tweets data frame, it has 6 columns and 2356 entries **

## Assess

After gathering each of the above pieces of data, assess them visually and programmatically for quality and tidiness issues will be our next step. We will detect and document at quality issues and tidiness issues.

In [16]:
# Print all archive dataset to assess it visually
archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [17]:
# Print some random expamles from columns values
# random number just to check if we can find something sucpisious
print(archive['text'][900])
print(archive['name'][12])

Meet Boston. He's worried because his tongue won't fit all the way in his mouth. 12/10 it'll be ok deep breaths pup https://t.co/rfWQ4T9iQj
None


In [18]:
# Assessing the data programmaticaly
archive.info()
archive.describe()
archive['rating_numerator'].value_counts()
archive['rating_denominator'].value_counts()
archive['name'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

None        745
a            55
Charlie      12
Lucy         11
Oliver       11
Cooper       11
Penny        10
Tucker       10
Lola         10
Winston       9
Bo            9
the           8
Sadie         8
Buddy         7
Daisy         7
Toby          7
Bailey        7
an            7
Oscar         6
Leo           6
Jack          6
Scout         6
Dave          6
Milo          6
Bella         6
Jax           6
Rusty         6
Koda          6
Stanley       6
Oakley        5
           ... 
Mike          1
Willie        1
Brooks        1
Clyde         1
Brownie       1
Philippe      1
Tobi          1
Florence      1
Obie          1
Maks          1
Bobbay        1
Fillup        1
Miley         1
Chase         1
Kaia          1
Gunner        1
Chesney       1
Sundance      1
Murphy        1
Kramer        1
Juckson       1
BeBe          1
Barclay       1
Meera         1
Remus         1
Milky         1
Pippin        1
Rhino         1
Rambo         1
Flash         1
Name: name, Length: 957,

In [19]:
images

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [99]:
images.info()
images['jpg_url'].value_counts()
images[images['jpg_url'] == 'https://pbs.twimg.com/media/CZhn-QAWwAASQan.jpg']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
800,691416866452082688,https://pbs.twimg.com/media/CZhn-QAWwAASQan.jpg,1,Lakeland_terrier,0.530104,True,Irish_terrier,0.197314,True,Airedale,0.082515,True
1624,803692223237865472,https://pbs.twimg.com/media/CZhn-QAWwAASQan.jpg,1,Lakeland_terrier,0.530104,True,Irish_terrier,0.197314,True,Airedale,0.082515,True


In [100]:
json_tweets
json_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 6 columns):
tweet_id           2356 non-null int64
favorites          2356 non-null int64
retweets           2356 non-null int64
user_followers     2356 non-null int64
user_favourites    2356 non-null int64
date_time          2356 non-null object
dtypes: int64(5), object(1)
memory usage: 110.5+ KB


### optional  

In [101]:
#json_tweets['tweet_id'].value_counts() #count tweet_ids

In [102]:
#json_tweets['user_followers'].value_counts() #check if querying the use_followers had a meaning

### Quality
*Completeness, Validity, Accuracy, Consistency => a.k.a content issues*

**`archive` dataset**
- `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` should be `intergs` instead of `float`
- `retweeted_status_timestamp`, `timestamp` should be `datetime` instead of `object` (string)
- The numerator and denominator columns have invalid values
- In several columns null objects are non-null (None to NaN)
- `Name` column have invalid names i.e 'None', 'a', 'an'
- We only want original ratings (no retweets) that have images
- We may want to change this columns type (`in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` and `tweet_id`) to `string` because We don't want any operations on them

**`images` dataset**
- Missing values from images dataset (2075 rows instead of 2356)
- Some tweet_ids have the same jpg_url
- Some tweets are have 2 different tweet_id one redirect to the other

**`json_tweeets` dataset**
- This tweet_id (666020888022790149) duplicated 8 times

### Tidiness
*Untidy data => a.k.a structural issues*
- No need to all the informations in `images dataset`, (tweet_id and jpg_url what matters)
- Various stages of dogs in columns instead of rows `archives dataset`
- We may want to add a gender column from the text columns in `archives dataset`
- All tables should be part of one dataset

## Clean

Cleaning our data is the third step in data wrangling. It is where we will fix the quality and tidiness issues that we identified in the assess step.

In [103]:
# Since we want to create one high quality and tidy master pandas DataFrame
# we will start by merging our dataframe in one
# then we save the result in file as backup
df_master = pd.merge(archive, images, how = 'left', on = ['tweet_id'] )
df_master = pd.merge(df_master, json_tweets, how = 'left', on = ['tweet_id'])
df_master.to_csv('df_master.csv', encoding = 'utf-8')
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2363 entries, 0 to 2362
Data columns (total 33 columns):
tweet_id                      2363 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2363 non-null object
source                        2363 non-null object
text                          2363 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2304 non-null object
rating_numerator              2363 non-null int64
rating_denominator            2363 non-null int64
name                          2363 non-null object
doggo                         2363 non-null object
floofer                       2363 non-null object
pupper                        2363 non-null object
puppo                         2363 non-null object
jpg_url                       20

We notice 7 additional rows, it maybe because of the sencond query for those tweet_id that tweeepy except throw. We will investigate on this more while cleaning our data.

### Clean rows and columns that we will not need

In [104]:
# Delete the retweets
df_master = df_master[pd.isnull(df_master.retweeted_status_id)]
# Delete duplicated tweet_id
df_master = df_master.drop_duplicates()
# Delete tweets with no pictures
df_master = df_master.dropna(subset = ['jpg_url'])

# test
len(df_master)

1994

In [105]:
# Delete columns related to retweet we don't need anymore
df_master = df_master.drop('retweeted_status_id', 1)
df_master = df_master.drop('retweeted_status_user_id', 1)
df_master = df_master.drop('retweeted_status_timestamp', 1)

# Delete column date_time we imported from the API, it has the same values as timestamp column
df_master = df_master.drop('date_time', 1)

# test
list(df_master)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'favorites',
 'retweets',
 'user_followers',
 'user_favourites']

### Melt the 'doggo', 'floofer', 'pupper' and 'puppo' columns into one column 'dog_stage'

In [106]:
# Check the values in those columns by excuting those columns
print('Doggo value\n')
print(df_master.doggo.value_counts())
print('\n--------------------------------\n')
print('Floofer value\n')
print(df_master.floofer.value_counts())
print('\n--------------------------------\n')
print('Pupper value\n')
print(df_master.pupper.value_counts())
print('\n--------------------------------\n')
print('Puppo value\n')
print(df_master.puppo.value_counts())

Doggo value

None     1920
doggo      74
Name: doggo, dtype: int64

--------------------------------

Floofer value

None       1986
floofer       8
Name: floofer, dtype: int64

--------------------------------

Pupper value

None      1782
pupper     212
Name: pupper, dtype: int64

--------------------------------

Puppo value

None     1971
puppo      23
Name: puppo, dtype: int64


In [107]:
# Select the columns to melt and to remain
columns_to_melt = ['doggo', 'floofer', 'pupper', 'puppo']
columns_to_stay = [x for x in df_master.columns.tolist() if x not in columns_to_melt]

# Mlet the the columns into values
df_master = pd.melt(df_master, id_vars = columns_to_stay, value_vars = columns_to_melt, 
                         var_name = 'stages', value_name = 'dog_stage')

# Delete column 'stages'
df_master = df_master.drop('stages', 1)

# Filter for unique values then remove duplicate values based on 'dog_stage' values

# This part for test *
print(df_master.dog_stage.value_counts())

df_master = df_master.sort_values('dog_stage').drop_duplicates('tweet_id', keep = 'last')

# This part for test
print(df_master.dog_stage.value_counts())
print(len(df_master))

None       7659
pupper      212
doggo        74
puppo        23
floofer       8
Name: dog_stage, dtype: int64
None       1688
pupper      212
doggo        63
puppo        23
floofer       8
Name: dog_stage, dtype: int64
1994


### Get rid of image prediction columns

In [108]:
# We will store the fisrt true algorithm with it's level of confidence
prediction_algorithm = []
confidence_level = []

# Get_prediction_confidence function:
# search the first true algorithm and append it to a list with it's level of confidence
# if flase prediction_algorthm will have a value of NaN
def get_prediction_confidence(dataframe):
    if dataframe['p1_dog'] == True:
        prediction_algorithm.append(dataframe['p1'])
        confidence_level.append(dataframe['p1_conf'])
    elif dataframe['p2_dog'] == True:
        prediction_algorithm.append(dataframe['p2'])
        confidence_level.append(dataframe['p2_conf'])
    elif dataframe['p3_dog'] == True:
        prediction_algorithm.append(dataframe['p3'])
        confidence_level.append(dataframe['p3_conf'])
    else:
        prediction_algorithm.append('NaN')
        confidence_level.append(0)

df_master.apply(get_prediction_confidence, axis=1)
df_master['prediction_algorithm'] = prediction_algorithm
df_master['confidence_level'] = confidence_level

# Test
list(df_master)


['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'favorites',
 'retweets',
 'user_followers',
 'user_favourites',
 'dog_stage',
 'prediction_algorithm',
 'confidence_level']

In [109]:
# Delete the columns of image prediction information
df_master = df_master.drop(['img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], 1)

# Test
list(df_master)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'jpg_url',
 'favorites',
 'retweets',
 'user_followers',
 'user_favourites',
 'dog_stage',
 'prediction_algorithm',
 'confidence_level']

### Check for the duplicated values and delete useless informations

In [110]:
# Print the count of the unique elements in all columns
df_master.apply(lambda x: len(x.unique()))

tweet_id                 1994
in_reply_to_status_id      23
in_reply_to_user_id         2
timestamp                1994
source                      3
text                     1994
expanded_urls            1994
rating_numerator           34
rating_denominator         15
name                      936
jpg_url                  1994
favorites                1874
retweets                 1624
user_followers            201
user_favourites             2
dog_stage                   5
prediction_algorithm      114
confidence_level         1684
dtype: int64

In [111]:
# let's concentrate on low values.. let's dig more
df_master.info()
df_master['in_reply_to_user_id'].value_counts()
df_master['source'].value_counts()
df_master['user_favourites'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 1918 to 6764
Data columns (total 18 columns):
tweet_id                 1994 non-null int64
in_reply_to_status_id    23 non-null float64
in_reply_to_user_id      23 non-null float64
timestamp                1994 non-null object
source                   1994 non-null object
text                     1994 non-null object
expanded_urls            1994 non-null object
rating_numerator         1994 non-null int64
rating_denominator       1994 non-null int64
name                     1994 non-null object
jpg_url                  1994 non-null object
favorites                1994 non-null float64
retweets                 1994 non-null float64
user_followers           1994 non-null float64
user_favourites          1994 non-null float64
dog_stage                1994 non-null object
prediction_algorithm     1994 non-null object
confidence_level         1994 non-null float64
dtypes: float64(7), int64(3), object(8)
memory usage: 296.0+ K

124451.0    1102
124450.0     892
Name: user_favourites, dtype: int64

The result of the above lines says:
- One value in **in_reply_to_user_id** so we will delete the columns of reply all of them replying to one user id '4196983835' @dog_rates
- **source** has 3 types, we will clean that column and made them clear
- **user_favourites** has 2 values and they are close, (that information meant to calculate the % of the like by favorites of the user; to see if the number of the folowers or the favorites of the page in general affect the number of the favorites or likes a tweet will get. Since this dataset gathered by querying the API in one date this information has no meaning. we will delete this column

In [112]:
# drop the following columns 'in_reply_to_status_id', 'in_reply_to_user_id', 'user_favourites'
df_master = df_master.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'user_favourites'], 1)

In [113]:
# Clean the content of source column
df_master['source'] = df_master['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])

# Test
df_master

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,favorites,retweets,user_followers,dog_stage,prediction_algorithm,confidence_level
1918,667405339315146752,2015-11-19 18:13:27 +0000,Twitter for iPhone,This is Biden. Biden just tripped... 7/10 http...,https://twitter.com/dog_rates/status/667405339...,7,10,Biden,https://pbs.twimg.com/media/CUMZnmhUEAEbtis.jpg,488.0,232.0,4484774.0,None,Saint_Bernard,0.381377
1917,667435689202614272,2015-11-19 20:14:03 +0000,Twitter for iPhone,Ermergerd 12/10 https://t.co/PQni2sjPsm,https://twitter.com/dog_rates/status/667435689...,12,10,None,https://pbs.twimg.com/media/CUM1OHCW4AEgGSi.jpg,324.0,88.0,4484774.0,None,Rottweiler,0.999091
1916,667437278097252352,2015-11-19 20:20:22 +0000,Twitter for iPhone,Never seen this breed before. Very pointy pup....,https://twitter.com/dog_rates/status/667437278...,10,10,None,https://pbs.twimg.com/media/CUM2qWaWoAUZ06L.jpg,478.0,255.0,4484774.0,None,NaN,0.000000
1915,667443425659232256,2015-11-19 20:44:47 +0000,Twitter for iPhone,Exotic dog here. Long neck. Weird paws. Obsess...,https://twitter.com/dog_rates/status/667443425...,6,10,None,https://pbs.twimg.com/media/CUM8QZwW4AAVsBl.jpg,823.0,616.0,4484774.0,None,NaN,0.000000
1914,667453023279554560,2015-11-19 21:22:56 +0000,Twitter Web Client,Meet Cupcake. I would do unspeakable things fo...,https://twitter.com/dog_rates/status/667453023...,11,10,Cupcake,https://pbs.twimg.com/media/CUNE_OSUwAAdHhX.jpg,327.0,95.0,4484774.0,None,Labrador_retriever,0.825670
1913,667455448082227200,2015-11-19 21:32:34 +0000,Twitter Web Client,This is Reese and Twips. Reese protects Twips....,https://twitter.com/dog_rates/status/667455448...,7,10,Reese,https://pbs.twimg.com/media/CUNHMXTU8AAS3HH.jpg,202.0,66.0,4484774.0,None,Tibetan_terrier,0.676376
1912,667470559035432960,2015-11-19 22:32:36 +0000,Twitter Web Client,This is a northern Wahoo named Kohl. He runs t...,https://twitter.com/dog_rates/status/667470559...,11,10,a,https://pbs.twimg.com/media/CUNU78YWEAECmpB.jpg,273.0,101.0,4484774.0,None,toy_poodle,0.304175
1911,667491009379606528,2015-11-19 23:53:52 +0000,Twitter Web Client,Two dogs in this one. Both are rare Jujitsu Py...,https://twitter.com/dog_rates/status/667491009...,7,10,None,https://pbs.twimg.com/media/CUNniSlUYAEj1Jl.jpg,556.0,237.0,4484774.0,None,borzoi,0.852088
1910,667495797102141441,2015-11-20 00:12:54 +0000,Twitter Web Client,This is Philippe from Soviet Russia. Commandin...,https://twitter.com/dog_rates/status/667495797...,9,10,Philippe,https://pbs.twimg.com/media/CUNr4-7UwAAg2lq.jpg,554.0,293.0,4484773.0,None,Chihuahua,0.143957
1909,667502640335572993,2015-11-20 00:40:05 +0000,Twitter Web Client,Say hello to Hall and Oates. Oates is winking ...,https://twitter.com/dog_rates/status/667502640...,11,10,Hall,https://pbs.twimg.com/media/CUNyHTMUYAAQVch.jpg,562.0,230.0,4484773.0,None,Labrador_retriever,0.996709


### Fill empty rating and Correct the bad ones

In [114]:
# Print the values and check if there exist in the text
df_master.rating_numerator.value_counts()
df_master.rating_denominator.value_counts()
print('\n')
print(df_master[df_master.rating_denominator == 170]['text'][2842])
print('\n-------------------------------------\n')

print(df_master[df_master.rating_numerator == 1776]['text'][2720])
print('\n-------------------------------------\n')

print(df_master[df_master.tweet_id == 786709082849828864]['text'][2497])
print('\n-------------------------------------\n')

print(df_master['text'][1918])
print('\n-------------------------------------\n')
print(df_master['text'][1917])
print('\n-------------------------------------\n')
print(df_master['text'][1916])
print('\n-------------------------------------\n')
print(df_master['text'][1911])
print('\n')



Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv

-------------------------------------

This is Atticus. He's quite simply America af. 1776/10 https://t.co/GRXwMxLBkh

-------------------------------------

This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS

-------------------------------------

This is Biden. Biden just tripped... 7/10 https://t.co/3Fm9PwLju1

-------------------------------------

Ermergerd 12/10 https://t.co/PQni2sjPsm

-------------------------------------

Never seen this breed before. Very pointy pup. Hurts when you cuddle. Still cute tho. 10/10 https://t.co/97HuBrVuOx

-------------------------------------

Two dogs in this one. Both are rare Jujitsu Pythagoreans. One slightly whiter than other. Long legs. 7/10 and 8/10 https://t.co/ITxxcc4v9y




- **It turns that the rating maybe thousands not only tens or hundreds**
- **Some tweets have more than one dog so more than one rating**
- **some tweets have no rating**
- **some dogs have float rating as 9.75**

In [115]:
# Get ratings and treat them depending to their situation
ratings = df_master['text'].apply(lambda x: re.findall(r'(\d+(\.\d+)|(\d+))\/(\d+0)', x))
len(ratings)

1994

In [116]:
# Add new columns to store the new ratings and the count of dogs in each tweet
rating_numerator = []
rating_denominator = []
dogs_count = []

for rate in ratings:
    # Tweets with no rating
    if len(rate) == 0:
        rating_numerator.append('NaN')
        rating_denominator.append('NaN')
        dogs_count.append(1) # It has a picture so it is a dog
    
    # Tweets with one rate
    elif len(rate) == 1:
        rating_numerator.append((float(rate[0][0]) / (float(rate[0][-1])/10)))
        rating_denominator.append(float(rate[0][-1]))
        dogs_count.append(float(rate[0][-1]) / 10) 
                                # We assume that the ratings who had rating_denominator different than 10
                                
    # we take the average of the tweet with more than one rating
    elif len(rate) > 1 and rate[0][-1] == '10':
        rating_plus = 0
        rating_avg = 0
        for i in range(len(rate)):
            rating_plus = rating_plus + float(rate[i][0])
        result_avg = (rating_plus / len(rate))
        rating_numerator.append(result_avg)
        rating_denominator.append(10)
        dogs_count.append(len(rate))
    else: # without this block I get ValueError: Length of values does not match length of index
          # We will try to catch the errors this  why and see why this happend
        rating_numerator.append('Error')
        rating_denominator.append('Error')
        dogs_count.append('Error')

df_master['new_rating_numerator'] = rating_numerator
df_master['new_rating_denominator'] = rating_denominator
df_master['dogs_count'] = dogs_count
df_master['new_rating_numerator'].value_counts()

12.0                 453
10.0                 411
11.0                 399
13.0                 261
9.0                  152
8.0                   94
7.0                   52
14.0                  36
6.0                   32
5.0                   30
3.0                   19
4.0                   14
2.0                    9
1.0                    4
8.5                    4
9.5                    3
7.5                    3
0.0                    2
10.5                   2
Error                  2
NaN                    1
1776.0                 1
4.5                    1
5.5                    1
6.5                    1
11.26                  1
13.5                   1
9.666666666666666      1
420.0                  1
11.5                   1
9.75                   1
11.27                  1
Name: new_rating_numerator, dtype: int64

In [117]:
# Test 2919 2885
print('\n')
print(df_master[df_master.new_rating_numerator == 'Error']['text'][2919])
print('\n-------------------\n')
print(df_master[df_master.new_rating_numerator == 'Error']['text'][2885])
print('\n-------------------\n')
print(df_master[df_master.new_rating_numerator == 1776.0]['text'][2720])
print('\n-------------------\n')
print(df_master[df_master.new_rating_numerator == 420.0]['text'][3712])
print('\n-------------------\n')



This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq

-------------------

Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a

-------------------

This is Atticus. He's quite simply America af. 1776/10 https://t.co/GRXwMxLBkh

-------------------

After so many requests... here you go.

Good dogg. 420/10 https://t.co/yfAAo1gdeY

-------------------



- **We notice that the two errors where for a special situation that has two fractions but only one of them is the valid one. we will could assess this manually and we will clean it manually as well.**
- **We still see two big numbers one of them for Atticus and the other for snop Dog we will exclude them in future visualisations**

In [118]:
# Correct the errors
tweet_id_11 = df_master[df_master.new_rating_numerator == 'Error']['tweet_id'][2919]
tweet_id_13 = df_master[df_master.new_rating_numerator == 'Error']['tweet_id'][2885]

df_master.loc[df_master['tweet_id'] == tweet_id_11, 'new_rating_numerator'] = 11
df_master.loc[df_master['tweet_id'] == tweet_id_13, 'new_rating_numerator'] = 13

df_master.loc[df_master['dogs_count'] == 'Error', 'dogs_count'] = 1
df_master.loc[df_master['new_rating_denominator'] == 'Error', 'new_rating_denominator'] = 10

# Test
print(df_master.new_rating_numerator[df_master.tweet_id == tweet_id_11])
print(df_master.new_rating_numerator[df_master.tweet_id == tweet_id_13])

2919    11
Name: new_rating_numerator, dtype: object
2885    13
Name: new_rating_numerator, dtype: object


In [119]:
# Delete the old columns and update the names of the new ones
df_master = df_master.drop(['rating_numerator', 'rating_denominator'], 1)

# Rename columns
df_master.rename(columns = {'new_rating_numerator': 'rating_numerator', 
                            'new_rating_denominator': 'rating_denominator'}, inplace = True)

# Test
list(df_master)

['tweet_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'name',
 'jpg_url',
 'favorites',
 'retweets',
 'user_followers',
 'dog_stage',
 'prediction_algorithm',
 'confidence_level',
 'rating_numerator',
 'rating_denominator',
 'dogs_count']

### Fill empty names and Correct the bad ones

The examples we could notice from assessing the data visually:
- This is [name] ..
- Meet [name] ..
- Say hallo to [name] ..
- Here we have [name] ..
- .. named [name] ..

We will treat those cases to get the names from the text of the tweet

In [120]:
# Loop on all the texts and check if the comment has one of the above conditions
# and append the result in a list
dog_names = []

for text in df_master['text']:
    # Start with 'This is ' and the fisrt letter of the name is uppercase
    if text.startswith('This is ') and re.match(r'[A-Z].*', text.split()[2]):
        dog_names.append(text.split()[2].strip(',').strip('.'))
    # Start with 'Meet ' and the fisrt letter of the name is uppercase
    elif text.startswith('Meet ') and re.match(r'[A-Z].*', text.split()[1]):
        dog_names.append(text.split()[1].strip(',').strip('.'))
    # Start with 'Say hello to ' and the fisrt letter of the name is uppercase
    elif text.startswith('Say hello to ') and re.match(r'[A-Z].*', text.split()[3]):
        dog_names.append(text.split()[3].strip(',').strip('.'))
    # Start with 'Here we have ' and the fisrt letter of the name is uppercase
    elif text.startswith('Here we have ') and re.match(r'[A-Z].*', text.split()[3]):
        dog_names.append(text.split()[3].strip(',').strip('.'))
    # Contain 'named' and the fisrt letter of the name is uppercase
    elif 'named' in text and re.match(r'[A-Z].*', text.split()[text.split().index('named') + 1]):
        dog_names.append(text.split()[text.split().index('named') + 1].strip(',').strip('.'))
    # No name specified or other style
    else:
        dog_names.append('NaN')

# Test
len(dog_names)

# Save the result in a new column 'dog_name'
df_master['dog_name'] = dog_names

# Test
print("New column dog_name count \n", df_master.dog_name.value_counts())
print("Old column name count \n", df_master.name.value_counts())

New column dog_name count 
 NaN           625
Charlie        11
Lucy           10
Oliver         10
Cooper         10
Penny           9
Tucker          9
Winston         8
Sadie           8
Daisy           7
Lola            7
Stanley         6
Toby            6
Bella           6
Koda            6
Bo              6
Jax             6
Louis           5
Chester         5
Milo            5
Buddy           5
Scout           5
Leo             5
Oscar           5
Bailey          5
Rusty           5
Sophie          4
Dexter          4
Duke            4
Brody           4
             ... 
Heinrich        1
Kaia            1
Beebop          1
Boots           1
Gromit          1
Kramer          1
Longfellow      1
Nico            1
Keurig          1
Randall         1
Rodney          1
Griswold        1
Kevon           1
Petrick         1
Fizz            1
Crimson         1
Glacier         1
Stu             1
Dylan           1
Godzilla        1
Harnold         1
Jo              1
Vixen           1


In [121]:
# We can see here that the 'NaN' result for the tweets with two names or no name
df_master[df_master.dog_name == 'NaN']

,tweet_id,timestamp,source,text,expanded_urls,name,jpg_url,favorites,retweets,user_followers,dog_stage,prediction_algorithm,confidence_level,rating_numerator,rating_denominator,dogs_count,dog_name
1917,667435689202614272,2015-11-19 20:14:03 +0000,Twitter for iPhone,Ermergerd 12/10 https://t.co/PQni2sjPsm,https://twitter.com/dog_rates/status/667435689...,None,https://pbs.twimg.com/media/CUM1OHCW4AEgGSi.jpg,324.0,88.0,4484774.0,None,Rottweiler,0.999091,12,10,1,NaN
1916,667437278097252352,2015-11-19 20:20:22 +0000,Twitter for iPhone,Never seen this breed before. Very pointy pup....,https://twitter.com/dog_rates/status/667437278...,None,https://pbs.twimg.com/media/CUM2qWaWoAUZ06L.jpg,478.0,255.0,4484774.0,None,NaN,0.000000,10,10,1,NaN
1915,667443425659232256,2015-11-19 20:44:47 +0000,Twitter for iPhone,Exotic dog here. Long neck. Weird paws. Obsess...,https://twitter.com/dog_rates/status/667443425...,None,https://pbs.twimg.com/media/CUM8QZwW4AAVsBl.jpg,823.0,616.0,4484774.0,None,NaN,0.000000,6,10,1,NaN
1911,667491009379606528,2015-11-19 23:53:52 +0000,Twitter Web Client,Two dogs in this one. Both are rare Jujitsu Py...,https://twitter.com/dog_rates/status/667491009...,None,https://pbs.twimg.com/media/CUNniSlUYAEj1Jl.jpg,556.0,237.0,4484774.0,None,borzoi,0.852088,7.5,10,2,NaN
1906,667524857454854144,2015-11-20 02:08:22 +0000,Twitter Web Client,Another topnotch dog. His name is Big Jumpy Ra...,https://twitter.com/dog_rates/status/667524857...,None,https://pbs.twimg.com/media/CUOGUfJW4AA_eni.jpg,1786.0,1188.0,4484773.0,None,Chesapeake_Bay_retriever,0.088122,12,10,1,NaN
1901,667546741521195010,2015-11-20 03:35:20 +0000,Twitter Web Client,Here is George. George took a selfie of his ne...,https://twitter.com/dog_rates/status/667546741...,George,https://pbs.twimg.com/media/CUOaOWXWcAA0_Jy.jpg,350.0,135.0,4484773.0,None,toy_poodle,0.787424,9,10,1,NaN
1900,667549055577362432,2015-11-20 03:44:31 +0000,Twitter Web Client,Never seen dog like this. Breathes heavy. Tilt...,https://twitter.com/dog_rates/status/667549055...,None,https://pbs.twimg.com/media/CUOcVCwWsAERUKY.jpg,6079.0,2438.0,4484773.0,None,NaN,0.000000,1,10,1,NaN
1899,667724302356258817,2015-11-20 15:20:54 +0000,Twitter Web Client,What a dog to start the day with. Very calm. L...,https://twitter.com/dog_rates/status/667724302...,None,https://pbs.twimg.com/media/CUQ7tv3W4AA3KlI.jpg,515.0,340.0,4484773.0,None,NaN,0.000000,7,10,1,NaN
1895,667782464991965184,2015-11-20 19:12:01 +0000,Twitter for iPhone,Super rare dog. Endangered (?). Thinks it's fu...,https://twitter.com/dog_rates/status/667782464...,None,https://pbs.twimg.com/media/CURwm3cUkAARcO6.jpg,433.0,257.0,4484773.0,None,NaN,0.000000,9,10,1,NaN
1894,667793409583771648,2015-11-20 19:55:30 +0000,Twitter for iPhone,Dogs only please. Small cows and other non can...,https://twitter.com/dog_rates/status/667793409...,None,https://pbs.twimg.com/media/CUR6jqVWsAEgGot.jpg,731.0,356.0,4484773.0,None,dalmatian,0.535073,8,10,1,NaN


In [122]:
# Let's delete the old name column now
df_master = df_master.drop(['name'], 1)

# Test
list(df_master)

['tweet_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'jpg_url',
 'favorites',
 'retweets',
 'user_followers',
 'dog_stage',
 'prediction_algorithm',
 'confidence_level',
 'rating_numerator',
 'rating_denominator',
 'dogs_count',
 'dog_name']

### Get Dogs gender column from text column

In [123]:
# Loop on all the text and check if it has one of pronouns of male or female
# and append the result in a list

male = ['He', 'he', 'him', 'his', "he's", 'himself']
female = ['She', 'she', 'her', 'hers', 'herself', "she's"]

dog_gender = []

for text in df_master['text']:
    # Male
    if any(map(lambda v:v in male, text.split())):
        dog_gender.append('male')
    # Female
    elif any(map(lambda v:v in female, text.split())):
        dog_gender.append('female')
    # If group or not specified
    else:
        dog_gender.append('NaN')

# Test
len(dog_gender)

# Save the result in a new column 'dog_name'
df_master['dog_gender'] = dog_gender

# Test
print("dog_gender count \n", df_master.dog_gender.value_counts())

dog_gender count 
 NaN       1132
male       636
female     226
Name: dog_gender, dtype: int64


### Convert the null values to None type

In [124]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 1918 to 6764
Data columns (total 17 columns):
tweet_id                1994 non-null int64
timestamp               1994 non-null object
source                  1994 non-null object
text                    1994 non-null object
expanded_urls           1994 non-null object
jpg_url                 1994 non-null object
favorites               1994 non-null float64
retweets                1994 non-null float64
user_followers          1994 non-null float64
dog_stage               1994 non-null object
prediction_algorithm    1994 non-null object
confidence_level        1994 non-null float64
rating_numerator        1994 non-null object
rating_denominator      1994 non-null object
dogs_count              1994 non-null object
dog_name                1994 non-null object
dog_gender              1994 non-null object
dtypes: float64(4), int64(1), object(12)
memory usage: 280.4+ KB


In [125]:
df_master.loc[df_master['prediction_algorithm'] == 'NaN', 'prediction_algorithm'] = None
df_master.loc[df_master['dog_name'] == 'NaN', 'dog_name'] = None
df_master.loc[df_master['dog_gender'] == 'NaN', 'dog_gender'] = None
df_master.loc[df_master['rating_numerator'] == 'NaN', 'rating_numerator'] = 0
df_master.loc[df_master['rating_denominator'] == 'NaN', 'rating_denominator'] = 0

In [126]:
# Test
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 1918 to 6764
Data columns (total 17 columns):
tweet_id                1994 non-null int64
timestamp               1994 non-null object
source                  1994 non-null object
text                    1994 non-null object
expanded_urls           1994 non-null object
jpg_url                 1994 non-null object
favorites               1994 non-null float64
retweets                1994 non-null float64
user_followers          1994 non-null float64
dog_stage               1994 non-null object
prediction_algorithm    1686 non-null object
confidence_level        1994 non-null float64
rating_numerator        1994 non-null object
rating_denominator      1994 non-null object
dogs_count              1994 non-null object
dog_name                1369 non-null object
dog_gender              862 non-null object
dtypes: float64(4), int64(1), object(12)
memory usage: 280.4+ KB


### Convert each column to its appropriate type

In [127]:
df_master.dtypes

tweet_id                  int64
timestamp                object
source                   object
text                     object
expanded_urls            object
jpg_url                  object
favorites               float64
retweets                float64
user_followers          float64
dog_stage                object
prediction_algorithm     object
confidence_level        float64
rating_numerator         object
rating_denominator       object
dogs_count               object
dog_name                 object
dog_gender               object
dtype: object

In [128]:
df_master['tweet_id'] = df_master['tweet_id'].astype(object)
df_master['timestamp'] = pd.to_datetime(df_master.timestamp)
df_master['source'] = df_master['source'].astype('category')
df_master['favorites'] = df_master['favorites'].astype(int)
df_master['retweets'] = df_master['retweets'].astype(int)
df_master['user_followers'] = df_master['user_followers'].astype(int)
df_master['dog_stage'] = df_master['dog_stage'].astype('category')
df_master['rating_numerator'] = df_master['rating_numerator'].astype(float)
df_master['rating_denominator'] = df_master['rating_denominator'].astype(float)
df_master['dogs_count'] = df_master['dogs_count'].astype(int)
df_master['dog_gender'] = df_master['dog_gender'].astype('category')

# Test
df_master.dtypes

tweet_id                        object
timestamp               datetime64[ns]
source                        category
text                            object
expanded_urls                   object
jpg_url                         object
favorites                        int32
retweets                         int32
user_followers                   int32
dog_stage                     category
prediction_algorithm            object
confidence_level               float64
rating_numerator               float64
rating_denominator             float64
dogs_count                       int32
dog_name                        object
dog_gender                    category
dtype: object

### Rename columns to be more expressive *and Clean if needed*

In [129]:
df_master = df_master.rename(columns = {'timestamp': 'tweet_date', 'source': 'tweet_source', 'text': 'tweet_text', 
                                        'expanded_urls': 'tweet_url', 'jpg_url': 'tweet_picture_predicted', 
                                        'favorites': 'tweet_favorites', 'retweets': 'tweet_retweets',
                                        'prediction_algorithm' : 'dog_breed'})

In [130]:
# All rating_denominator has one value 10
# We will delete this column
print(df_master.rating_denominator.value_counts())
df_master.drop('rating_denominator', 1, inplace = True)

10.0     1981
80.0        2
50.0        2
170.0       1
150.0       1
130.0       1
70.0        1
0.0         1
120.0       1
40.0        1
90.0        1
110.0       1
Name: rating_denominator, dtype: int64


In [131]:
# Test
list(df_master)

['tweet_id',
 'tweet_date',
 'tweet_source',
 'tweet_text',
 'tweet_url',
 'tweet_picture_predicted',
 'tweet_favorites',
 'tweet_retweets',
 'user_followers',
 'dog_stage',
 'dog_breed',
 'confidence_level',
 'rating_numerator',
 'dogs_count',
 'dog_name',
 'dog_gender']

## Storing, Analyzing, and Visualizing Data

In [132]:
# Store the clean DataFrame in a CSV file
df_master.to_csv('twitter_archive_master.csv', index=False, encoding = 'utf-8')

In [133]:
df_master = pd.read_csv('twitter_archive_master.csv')
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 16 columns):
tweet_id                   1994 non-null int64
tweet_date                 1994 non-null object
tweet_source               1994 non-null object
tweet_text                 1994 non-null object
tweet_url                  1994 non-null object
tweet_picture_predicted    1994 non-null object
tweet_favorites            1994 non-null int64
tweet_retweets             1994 non-null int64
user_followers             1994 non-null int64
dog_stage                  1994 non-null object
dog_breed                  1686 non-null object
confidence_level           1994 non-null float64
rating_numerator           1994 non-null float64
dogs_count                 1994 non-null int64
dog_name                   1369 non-null object
dog_gender                 862 non-null object
dtypes: float64(2), int64(5), object(9)
memory usage: 249.3+ KB


**The visualizations in the act_report.ipynd notebook**